In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import numpy.matlib
import matplotlib.pyplot as plt
import struct
from fetutorialfunctions import extractListedParamsFromFile

# Generate the WOA hill terrain file

In [ ]:
path_case = "INSERT_PATH_TO_YOUR_RUN_DIRECTORY"
parameters_file = path_case + 'Example07_DISPERSION_SBL.in'

In [ ]:
parameters_list = ['Nx','Ny','d_xi','d_eta']
pDict = extractListedParamsFromFile(parameters_file, parameters_list)
Nx = pDict['Nx']
Ny = pDict['Ny']
dx = pDict['d_xi']
dy = pDict['d_eta']

In [ ]:
xPos_2d = np.tile(np.linspace(0.5*dx,(Nx-0.5)*dx,Nx,endpoint=True),(Ny,1))
yPos_2d = np.tile(np.linspace(0.5*dy,(Ny-0.5)*dy,Ny,endpoint=True),(Nx,1)).T

In [ ]:
x0 = 0.5*(xPos_2d[0,-1]-xPos_2d[0,0])
y0 = 0.5*(yPos_2d[-1,0]-yPos_2d[0,0])
print('x0,y0=',x0,y0)

In [ ]:
y_diff = np.abs(yPos_2d[:,0]-y0)
y_ind = np.where(y_diff==np.min(y_diff))[0][0]
y0 = yPos_2d[y_ind,0]
print('y_ind=',y_ind)
print('y0=',y0)

In [ ]:
# Define the WOA hill
h0 = 200.0
ax = 30.0
ay = 30.0 
woa_topo = h0/(1.0 + (xPos_2d - x0)**2.0/(ax*ax) + (yPos_2d - y0)**2.0/(ay*ay))

In [ ]:
ridge_len_h = 25
woa_topo_ridge = np.zeros(woa_topo.shape)
woa_c_1d = woa_topo[y_ind,:]
print('woa_c_1d.shape=',woa_c_1d.shape)
woa_c_2d = np.matlib.repmat(woa_c_1d, 2*ridge_len_h+1, 1)
print('woa_c_2d.shape=',woa_c_2d.shape)
woa_topo_ridge[y_ind-ridge_len_h:y_ind+ridge_len_h+1,:] =  woa_c_2d
woa_topo_ridge[0:y_ind-ridge_len_h,:] = woa_topo[y_ind-(y_ind-ridge_len_h):y_ind,:]
woa_topo_ridge[y_ind+ridge_len_h+1:Ny,:] = woa_topo[y_ind+1:y_ind+(y_ind-ridge_len_h)+2,:]

In [ ]:
fntSize=14 # 20
plt.rcParams['xtick.labelsize']=fntSize
plt.rcParams['ytick.labelsize']=fntSize
plt.rcParams['axes.linewidth']=1.25
fntSize_legend = 10.0
fntSize_title = 16.0
markerSize = 7.0

fig,axs = plt.subplots(1,1,sharey=False,sharex=False,figsize=(6,5))
ax=axs
im=ax.pcolormesh(xPos_2d,yPos_2d,woa_topo_ridge,cmap='nipy_spectral' ,shading='auto') 

cbar=fig.colorbar(im, ax=ax,orientation='vertical')

ax.set_xlabel('x [m]',fontsize=fntSize)
ax.set_ylabel('y [m]',fontsize=fntSize)

In [ ]:
FE_new_terrain = f"{path_case}Topography_{Nx:d}x{Ny:d}.dat"
topoflat = bytearray(Nx*Ny*4)
struct.pack_into(f"{Nx*Ny:d}f",topoflat,0,*woa_topo_ridge.flatten())
topoFile = open(FE_new_terrain,'wb')
topoFile.write(struct.pack('i',Nx))
topoFile.write(struct.pack('i',Ny))
topoFile.write(topoflat)
topoFile.close()

In [ ]:
FE_new_terrain